In [17]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
type(documents)

list

In [57]:
import random
random.seed(1234)
small_docs = random.sample(documents, 200)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(small_docs)
tf_feature_names = tf_vectorizer.get_feature_names()

In [59]:
tf

<200x2163 sparse matrix of type '<class 'numpy.int64'>'
	with 7903 stored elements in Compressed Sparse Row format>

In [60]:
tf[0]

<1x2163 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [61]:
from sklearn.decomposition import LatentDirichletAllocation


no_topics = 10
lda = LatentDirichletAllocation(no_topics, max_iter=20, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


In [62]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[-no_top_words:]][::-1]))

no_top_words = 5
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
law jesus christ god faith
Topic 1:
course training applications esa data
Topic 2:
card drive game remove board
Topic 3:
good federal new coach defense
Topic 4:
team players edu nhl teams
Topic 5:
israeli human jews israel rights
Topic 6:
cx c_ ah ck 34u
Topic 7:
weapon models mouse copy ibm
Topic 8:
don just use like time
Topic 9:
gun right guns president new


In [74]:
import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import spacy
nlp = spacy.load('en_core_web_md')

random.seed(1234)
small_docs = random.sample(documents, 1000)


import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = [t.lower_ for t in nlp(text)]
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens
    
    
small_docs = [prepare_text_for_lda(text) for text in small_docs]

bigram = gensim.models.Phrases(small_docs)
small_docs_bigram = [bigram[t] for t in small_docs]

# create dictionary and corpus
dictionary = Dictionary(small_docs_bigram)

# corpus = (token_id, count_in_curr_doc) , sparse representation
corpus = [dictionary.doc2bow(d) for d in small_docs_bigram]
small_docs_id = [dictionary.doc2idx(document=tw) for tw in small_docs_bigram]
small_docs_id = [d for d in small_docs_id if len(d) != 0]

[nltk_data] Downloading package wordnet to /home/simi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/simi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## LDA Lecture With Pyro
First lets try the LDA model from the lecture

In [75]:
import pyro
import pyro.distributions as p
from pyro import optim
from pyro import infer
import torch
from torch.distributions import constraints

D = 200
N = [len(d) for d in small_docs_id]
K = 5
V = max([max(d) for d in small_docs_id])
print(f'D: {D} | V: {V} | K: {K}')

#@pyro.poutine.broadcast
def model(data):
    # topic - word distribution
    phi = pyro.sample("phi", p.Dirichlet(torch.ones([K, V])).independent(1))
  
    for d in pyro.plate("documents", D):
        # document-topic distribution
        theta_d = pyro.sample(f"theta_{d}", p.Dirichlet(torch.ones([K])))
    
        with pyro.plate(f"words_{d}", N[d]):
            z = pyro.sample(f"z_{d}", p.Categorical(theta_d))
            pyro.sample(f"w_{d}", p.Categorical(phi[z]), obs=data[d])
            
#@pyro.poutine.broadcast
def guide(data):
    beta_q = pyro.param("beta_q", torch.ones([K, V]),constraint=constraints.positive)
    phi_q = pyro.sample("phi",p.Dirichlet(beta_q).independent(1))
  
    for d in pyro.plate("documents", D):
        alpha_q = pyro.param(f"alpha_q_{d}", torch.ones([K]),constraint=constraints.positive)
        q_theta_d = pyro.sample(f"theta_{d}", p.Dirichlet(alpha_q))
    
        with pyro.plate(f"words_{d}", N[d]):
            q_i = pyro.param(f"q_{d}", torch.randn([N[d], K]).exp(), constraint=constraints.simplex)
            pyro.sample(f"z_{d}", p.Categorical(q_i))

data = [torch.tensor(d).float() for d in small_docs_id]            
            
adam_params = {"lr": 0.01, "betas": (0.90, 0.999)}
optimizer = optim.Adam(adam_params)

# infer.config_enumerate(guide, 'parallel')
pyro.clear_param_store()
svi = infer.SVI(model, guide, optimizer, loss=infer.TraceEnum_ELBO(max_iarange_nesting=1))
for _ in range(3000):
    loss = svi.step(data)
    print(loss)
        

D: 200 | V: 16326 | K: 5
166866.890625
166386.875
168104.78125
165847.75
165793.125
166837.015625
164976.984375
167028.09375
165783.796875
166757.609375
165319.15625
166916.578125
164538.46875
165873.53125
164800.484375
165610.46875
164444.9375
164544.859375
164338.84375
164269.953125
164790.265625
164373.78125
164519.703125
165452.640625
163850.5625
166307.984375
164315.34375
163745.71875
164122.40625
163247.21875
162860.21875
162194.8125
166975.96875
164064.71875
162706.078125
164393.953125
162885.25
163099.328125
165604.1875
164334.515625
161271.34375
163686.234375
163902.59375
162178.859375
162259.3125
162503.625
162396.6875
162566.265625
162065.03125
161936.5
163406.0625
164640.171875
160493.359375
161663.984375
161280.125
161069.734375
161232.640625
162113.671875
161480.515625
162347.9375
163308.015625
162554.859375
161784.328125
161206.03125
160951.46875
162687.9375
161369.953125
161321.515625
160351.25
160398.734375
161786.953125
161441.03125
161849.796875
160987.359375
160719.

152796.3125
153334.65625
153163.390625
153673.9375
152882.65625
153870.9375
153318.34375
152588.046875
152585.046875
152842.421875
154262.703125
152636.859375
153133.59375
152898.828125
153462.78125
152927.8125
152882.109375
153122.234375
153166.171875
152835.9375
153456.296875
153026.84375
152960.890625
152912.375
152893.5625
152716.328125
153115.40625
153189.0625
153251.25
153098.8125
152965.21875
153285.046875
152933.65625
153255.875
152703.578125
152801.453125
153089.828125
152845.875
153155.8125
153129.6875
152978.390625
153116.109375
152942.265625
152884.25
153132.484375
153029.921875
153030.78125
153232.640625
152891.234375
152813.546875
152899.34375
153031.078125
153052.421875
152979.953125
153369.734375
152878.0
152945.09375
153107.21875
152641.140625
152978.0625
153021.578125
152308.09375
152810.453125
152700.5625
152952.4375
152884.640625
152972.25
152979.609375
152805.078125
153010.03125
152397.109375
152833.9375
152831.640625
152766.203125
152732.265625
152901.921875
15342

151781.765625
152052.90625
151562.625
151910.703125
152143.328125
151659.1875
151997.46875
152108.296875
151823.515625
151801.328125
151734.8125
151592.140625
151926.703125
151846.125
151959.0
151862.734375
151766.859375
151667.8125
152030.203125
151540.875
151665.03125
151756.859375
151716.625
151842.96875
151829.875
151930.359375
152027.9375
151880.296875
151811.78125
151860.140625
151742.84375
152032.265625
151695.0625
151675.015625
151859.53125
151778.40625
151675.375
151757.71875
151817.84375
151512.96875
151934.75
151775.34375
151524.75
151924.703125
152054.984375
151953.890625
151738.234375
151615.453125
151381.125
151491.8125
151667.09375
151961.515625
151510.1875
151778.296875
152356.703125
151727.9375
151569.65625
151833.109375
151899.21875
151791.71875
151751.3125
151676.53125
151743.546875
151490.109375
151819.140625
151642.171875
151602.5
151728.234375
151726.671875
151760.203125
151891.171875
151755.9375
152032.125
151581.90625
151767.234375
151527.578125
151813.609375
15

151077.671875
151190.171875
151142.734375
151218.09375
151297.125
151275.28125
151137.125
151229.5625
151104.375
151404.25
151390.8125
151317.71875
150914.484375
151068.859375
151249.21875
151256.140625
150992.234375
151330.671875
151075.265625
151562.515625
150961.5
151209.859375
151031.171875
151464.546875
151285.46875
151230.078125
151282.203125
151097.234375
151178.0
151281.21875
151295.65625
151322.46875
151220.40625
151335.078125
151332.65625
151162.734375
151161.140625
151245.09375
151103.578125
151135.828125
150998.25
151258.359375
151205.421875
151310.53125
151417.5625
150913.953125
151140.296875
151433.046875
151046.796875
151227.515625
151108.65625
151559.1875
151247.296875
151244.375
151200.375
151104.34375
151131.671875
151326.3125
151118.640625
151406.546875
151316.703125
151111.71875
151123.140625
151110.515625
151263.40625
151147.78125
151127.59375
151165.984375
151114.734375
151300.828125
151136.21875
151168.890625
151226.9375
151455.078125
150944.09375
151161.953125
1

150779.25
150962.109375
151116.359375
150784.46875
150770.25
151065.4375
150970.9375
150965.765625
150895.15625
150809.40625
150745.0625
150993.03125
150765.359375
150930.390625
150806.234375
150809.3125
150827.515625
150992.9375
150717.25
150993.578125
150927.1875
150745.546875
150866.296875
150864.25
150607.71875
150899.046875
151044.375
150841.40625
150971.1875
150925.1875
150847.0
150911.125
151121.125
150771.765625
151211.84375
150760.671875
150963.546875
150870.78125
150820.265625
150894.703125
150938.296875
150748.9375
151087.375
150807.140625
151085.328125
150959.671875
150926.5625
150989.6875
150739.390625
150852.171875
150959.390625
150884.1875
150732.28125
150842.109375
150637.15625
150825.359375
151073.75
150756.78125
151188.90625
150667.921875
150930.9375
150875.109375
150815.0625
151056.578125
150891.625
150946.453125
150919.59375
150875.796875
151075.3125
150918.546875
150761.15625
150726.796875
150871.375
150751.203125
150860.03125
150870.125
150727.734375
150994.265625

In [76]:
params = pyro.get_param_store()

beta_q = params["beta_q"]
topic_words_distribution = p.Dirichlet(beta_q).sample()


for t in range(K):
    print("---- topic {} -----".format(t))
    print("  -- top words ---")
    top5_words = (torch.argsort(topic_words_distribution[t])[-10:]).cpu().numpy()
    top5_words = list(map(lambda x: dictionary[x], reversed(
        top5_words)))
    print(top5_words)    

---- topic 0 -----
  -- top words ---
['would', 'attack', '\n        ', 'think', 'm"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@', 'm"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@_m"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@', 'input', 'right', 'plaintext', 'cards']
---- topic 1 -----
  -- top words ---
['would', '\n    ', 'forgery', 'cryptosystem', 'provide', 'thing', 'think', 'system', 'seem', 'suppose']
---- topic 2 -----
  -- top words ---
['would', 'people', 'think', 'month', 'windows', 'support', 'information', 'right', 'advice', 'attack']
---- topic 3 -----
  -- top words ---
['would', 'right', '\n        ', 'technology', 'people', 'could', 'point', 'attack', 'wretched', 'computer']
---- topic 4 -----
  -- top words ---
['would', 'program', 'reward', 'case', 'right', 's/$5%13p\\eg,\\/#p\\t3qs', 'think', 'consider', 'reason', 'windows']


## LDA Proposed by Pyro

In [78]:
data_5 = torch.stack([t[:5] for t in data if len(t) >= 5])
data_5.shape

torch.Size([918, 5])

In [81]:

import torch
from torch import nn
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO
from pyro.optim import Adam

import functools
from operator import itemgetter

K = 5
V = len(dictionary)
V_per_doc = data_5.shape[1]
D = data_5.shape[0]
l_sizes = '100-100'

# This is a fully generative model of a batch of documents.
# data is a [num_words_per_doc, num_documents] shaped array of word ids
# (specifically it is not a histogram). We assume in this simple example
# that all documents have the same number of words.
def model(data=None, batch_size=None):
    # Globals.
    with pyro.plate("topics", K):
        topic_weights = pyro.sample("topic_weights", dist.Gamma(1. / K, 1.))
        topic_words = pyro.sample("topic_words", dist.Dirichlet(torch.ones(V) / V))

    # Locals.
    with pyro.plate("documents", D) as ind:
        if data is not None:
            with pyro.util.ignore_jit_warnings():
                assert data.shape == (V_per_doc, D)
            data = data[:, ind]

        doc_topics = pyro.sample("doc_topics", dist.Dirichlet(topic_weights))
        with pyro.plate("words", V_per_doc):
            # The word_topics variable is marginalized out during inference,
            # achieved by specifying infer={"enumerate": "parallel"} and using
            # TraceEnum_ELBO for inference. Thus we can ignore this variable in
            # the guide.
            word_topics = pyro.sample("word_topics", dist.Categorical(doc_topics),
                                      infer={"enumerate": "parallel"})
            data = pyro.sample("doc_words", dist.Categorical(topic_words[word_topics]),
                               obs=data)

    return topic_weights, topic_words, data


# We will use amortized inference of the local topic variables, achieved by a
# multi-layer perceptron. We'll wrap the guide in an nn.Module.
def make_predictor():
    layer_sizes = ([V] +
                   [int(s) for s in l_sizes.split('-')] +
                   [K])
    print('Creating MLP with sizes {}'.format(layer_sizes))
    layers = []
    for in_size, out_size in zip(layer_sizes, layer_sizes[1:]):
        layer = nn.Linear(in_size, out_size)
        layer.weight.data.normal_(0, 0.001)
        layer.bias.data.normal_(0, 0.001)
        layers.append(layer)
        layers.append(nn.Sigmoid())
    layers.append(nn.Softmax(dim=-1))
    return nn.Sequential(*layers)


def parametrized_guide(predictor, data, batch_size=None):
    # Use a conjugate guide for global variables.
    topic_weights_posterior = pyro.param(
        "topic_weights_posterior",
        lambda: torch.ones(K),
        constraint=constraints.positive)
    topic_words_posterior = pyro.param(
        "topic_words_posterior",
        lambda: torch.ones(K, V),
        constraint=constraints.greater_than(0.5))
    with pyro.plate("topics", K):
        pyro.sample("topic_weights", dist.Gamma(topic_weights_posterior, 1.))
        pyro.sample("topic_words", dist.Dirichlet(topic_words_posterior))

    # Use an amortized guide for local variables.
    pyro.module("predictor", predictor)
    with pyro.plate("documents", D, batch_size) as ind:
        # The neural network will operate on histograms rather than word
        # index vectors, so we'll convert the raw data to a histogram.
        if torch._C._get_tracing_state():
            counts = torch.eye(1024)[data[:, ind]].sum(0).t()
        else:
            counts = torch.zeros(V, ind.size(0))
            counts.scatter_add_(0, data[:, ind], torch.tensor(1.).expand(counts.shape))
        doc_topics = predictor(counts.transpose(0, 1))
        pyro.sample("doc_topics", dist.Delta(doc_topics, event_dim=1))


pyro.set_rng_seed(0)
pyro.clear_param_store()
#pyro.enable_validation(True)

# We can generate synthetic data directly by calling the model.
#true_topic_weights, true_topic_words, data = model()

# We'll train using SVI.
predictor = make_predictor()
guide = functools.partial(parametrized_guide, predictor)
Elbo = TraceEnum_ELBO  # JitTraceEnum_ELBO if args.jit else TraceEnum_ELBO
elbo = Elbo(max_plate_nesting=2)
optim = Adam({'lr': 1e-2})
svi = SVI(model, guide, optim, elbo)
print('Step\tLoss')
for step in range(1750):
    loss = svi.step(data_5.transpose(1,0).long(), batch_size=64)
    if step % 10 == 0:
        print('{: >5d}\t{}'.format(step, loss))

Creating MLP with sizes [16327, 100, 100, 5]
Step	Loss
    0	709365.75
   10	708818.0
   20	704644.5625
   30	700923.8125
   40	699250.75
   50	702152.5625
   60	696595.125
   70	696472.5
   80	692513.625
   90	689599.625
  100	690201.125
  110	685354.9375
  120	685191.375
  130	681636.4375
  140	683272.5
  150	682579.25
  160	681153.9375
  170	680963.0
  180	680103.125
  190	679101.0
  200	676949.4375
  210	677046.9375
  220	679176.5
  230	674253.25
  240	672848.625
  250	672646.6875
  260	672832.125
  270	671236.0625
  280	672293.8125
  290	671966.3125
  300	671197.375
  310	672278.3125
  320	670408.0
  330	670256.375
  340	670111.125
  350	669919.875
  360	670259.5625
  370	669315.9375
  380	672476.4375
  390	667629.75
  400	668559.4375
  410	671382.0
  420	671521.25
  430	668198.875
  440	668555.25
  450	669791.75
  460	667272.375
  470	665881.0
  480	667450.4375
  490	666210.75
  500	670156.4375
  510	665281.125
  520	667196.1875
  530	667379.75
  540	664072.25
  550	664735.5625
 

In [82]:
params = pyro.get_param_store()

words_per_topic_distr = params['topic_words_posterior']

for t in range(K):
    print("---- topic {} -----".format(t))
    top5_words = (torch.argsort(words_per_topic_distr[t])[-10:]).cpu().numpy()
    top5_words = list(map(lambda x: dictionary[x], reversed(top5_words)))
    print(top5_words)

---- topic 0 -----
['anyone', 'would', 'post', 'looking', 'system', 'first', 'please', 'hello', 'article', 'several']
---- topic 1 -----
['think', 'people', 'anyone', 'could', 'would', 'please', 'interest', 'looking', 'believe', 'point']
---- topic 2 -----
['would', 'people', 'could', 'anyone', 'think', 'please', 'someone', 'try', 'believe', 'thought']
---- topic 3 -----
['could', 'people', 'someone', 'following', 'would', 'windows', 'try', 'drive', 'really', 'right']
---- topic 4 -----
['think', 'would', 'people', 'could', 'someone', 'anyone', 'question', 'drive', 'article', 'believe']


## LDA On Twitter 

In [83]:
###### Twitter Data
#####################
#     LOAD DATA     #
#####################

import json_lines
import csv

def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['full_text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    d['created_at'] = tweet['created_at']
    return d

if False:
    with open('congress_dataset/senators-1.jsonl', 'rb') as f:
        with open(r'senators-1-tweets.csv', 'a') as file:
            writer = csv.writer(file)
            for item in json_lines.reader(f):
                # Only collect tweets in English
                if item['lang'] == 'en' and len(item['entities']['hashtags']) > 0:
                    tweet_data = process_tweet(item)
                    writer.writerow(list(tweet_data.values()))

                    
import pandas as pd
tweets = pd.read_csv("senators-1-tweets.csv", header=None, names=['hashtags', 'text', 'user', 'user_location', 'created_at'])  
print('num tweets: {}'.format(len(tweets)))


import spacy
nlp = spacy.load('en_core_web_md')

def tokenize(text):
    lda_tokens = []
    tokens = nlp(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens
    

import random

docs = []
hashtags = []
N = 2000
rand_tweets = list(range(N)) #random.sample(range(len(tweets)), k=N)
for i, tw in enumerate(rand_tweets):
    if i % 1000 == 0:
        print('{}%'.format(100./N*i), end=' ')
    text = tweets.iloc[i]['text']
    tokens = prepare_text_for_lda(text)
    if random.random() > .9999:
        print(tokens)
    taggs = tweets.iloc[i]['hashtags'].replace('[', '').replace(']', '').replace('\'', '').split(",")
    hashtags.append([t.strip() for t in taggs])
    docs.append(tokens)

num tweets: 449334


[nltk_data] Downloading package wordnet to /home/simi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/simi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.0% 50.0% ['forward', 'hearing', 'tillerson', 'mattis', 'potus', 'afghanistan', 'strategy', 'america', 'safe']


In [84]:
all_docs = [' '.join(d) for d in docs]

## LDA With Sklearn (Twitter)

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(all_docs)
tf_feature_names = tf_vectorizer.get_feature_names()

In [87]:
from sklearn.decomposition import LatentDirichletAllocation


no_topics = 10
lda = LatentDirichletAllocation(no_topics, max_iter=15, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


In [88]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
screen_name today health getcovered great deadline visit tomorrow question healthcare
Topic 1:
sesta solutionssummit2017 traffic accountable event trafficker company collaboration online enable
Topic 2:
county taxreform screen_name worker gardnerfarmtour alsen official increase income update
Topic 3:
matter thought means angusontheroad chair kitchentableissues operations price influence foreign
Topic 4:
family celebrate years happy friend school hanukkah life sandyhook today
Topic 5:
country american business economy support family woman goptaxscam trump right
Topic 6:
dreamer dreamactnow protect screen_name congress young americandreamer community 000 status
Topic 7:
screen_name netneutrality internet goptaxscam fight republican child savenetneutrality senate american
Topic 8:
screen_name today watch great utpol honor thanks taxreform discus community
Topic 9:
screen_name thank dreamer standing democrat family earn solidarity rally peace
